In [ ]:
#### IMPORTS ####
using ProgressMeter
using Plots 
using Measures
using LaTeXStrings
using DelimitedFiles
# using NPZ
include("source/mOrbRG.jl")
include("source/helpers.jl")
include("source/probes.jl")
const SIZE = (750,300)
scale = 1
discreteCgrad = cgrad(:BuPu_3, 3, categorical = true, rev=true)

In [ ]:
function getRgFlow(num_kspace_half, J_val, W_by_J_range, orbitals)
    num_kspace = 2 * num_kspace_half + 1
    k_vals = collect(range(K_MIN, K_MAX, length=num_kspace)) ./ pi
    for orbital in orbitals
        anim = @animate for (j, W_by_J) in collect(enumerate(W_by_J_range))
            savePath = "data/$(orbital)_$(num_kspace_half)_$(round(J_val, digits=3))_$(round(W_by_J, digits=3))"
            if !isfile(savePath)
                kondoJArray, dispersionArray = main(num_kspace_half, J_val, J_val * W_by_J, orbital)
                results = getGlobalFlow(kondoJArray, num_kspace_half, num_kspace_half, savePath)
                results_bool = getGlobalFlowBool(kondoJArray, num_kspace_half, num_kspace_half, savePath*"-bool")
            else
                results = readdlm(savePath)
                results_bool = readdlm(savePath*"-bool")
            end
            hm1 = heatmap(k_vals, k_vals, results_bool, 
                cmap=discreteCgrad, colorbar_ticks=[1, 0, -1])
            hm2 = heatmap(log10.(results), cmap=:diverging_bkr_55_10_c35_n256)
            p = plot(hm1, hm2, size=SIZE, 
                plot_title="\$W/J=$(round(W_by_J, digits=3))\$", 
                top_margin=3mm, left_margin=[5mm 10mm], bottom_margin=5mm,
                xlabel="\$ \\mathrm{k_x/\\pi} \$", ylabel="\$ \\mathrm{k_y/\\pi} \$",
                dpi=100 * scale
            )
        end
        gif(anim, "kondoUb_kspaceRG_$(orbital)_wave_$(num_kspace)_$(SIZE[1] * scale)x$(SIZE[2] * scale).gif", fps=2)
        gif(anim, "kondoUb_kspaceRG_$(orbital)_wave_$(num_kspace)_$(SIZE[1] * scale)x$(SIZE[2] * scale).mp4", fps=2)
    end

end

getRgFlow(4, 0.2, -0.01:-0.001:-0.05, ["p"])